# **Employee Shift Scheduling A Constraints Satisfiction problem**

You are tasked with developing an automated scheduling system that assigns employees to work shifts over a given period (e.g., one week). The assignment must satisfy several constraints to ensure fairness and operational efficiency. 


### **Objectives**
- **CSP Modeling:** Formulate the scheduling problem as a CSP by defining variables, domains, and constraints.
- **Search Techniques:** Implement search strategies such as backtracking, along with heuristics like Minimum Remaining Value (MRV) and LCV Constraining Variable (LCV).
- **Constraint Propagation:** Use techniques such as AC-3 to reduce the domain of possibilities for each shift before and during search.

### **Problem Description**:

**Schedule Structure**
- **Time Frame:** Consider a schedule spanning 5 days (Monday through Friday).
- **Shifts per Day:** Each day is divided into 3 shifts: Morning, Afternoon, and Night.
- **Employees:** You have a fixed list of employees available for scheduling.

### **Inputs**
1. **List of Employees:** An array of employee names (e.g., ["Alice", "Bob", "Carol", "Dave"]).
2. **Employee Availability:**: For each employee, a dictionary indicating the shifts or days they are available to work. 

### **Constraints**
1. **Daily Work Limit:** Each employee can work at most one shift per day.

2. **Availability Constraint:** An employee can only be assigned to a shift if they are available for that day/shift.

### **Expected Output**
- **Schedule Grid:** A 5×3 grid (or equivalent structure) where each cell shows the assigned employee for that day and shift.

    For example:

    | Day     | Morning | Afternoon | Night |
    |---------|---------|-----------|-------|
    | Monday  | Alice   | Carol     | Dave  |
    | Tuesday | Carol   | Bob       | Dave   |
    | Wednesday | ...   | ...       | ...   |
    | Thursday  | ...   | ...       | ...   |
    | Friday    | ...   | ...       | ...   |

### **Feasibility:** 
The produced schedule must satisfy all the constraints:
- No employee is scheduled more than once per day.
- Only available employees are assigned to shifts.


### **Overall Process**

1. **For Each Day in the Week:**
   - **Initialize Daily Variables and Domains:**
     - Build a `daySchedule` mapping each shift (e.g., Morning, Afternoon, Night) to an employee assignment (initially set to `None`).
     - Build `dayDomains` for each shift based on employee availability for that day.
   - **Run AC-3 for Constraint Propagation:**
     - Apply AC-3 to prune the domains of each shift using the "one shift per day" rule (i.e., no employee can work more than one shift per day).
   - **Perform Backtracking Search:**
     - **Select the Next Variable (Shift) using MRV:**  
       Choose the shift with the smallest remaining domain.
     - **Order Possible Assignments using LCV:**  
       Prioritize employee assignments that eliminate the fewest options for other shifts.
     - **Assign Employees and Check Consistency:**  
       Assign an employee to the chosen shift and verify that the assignment does not violate the constraints.
     - **Backtracking:**  
       If a dead-end occurs (i.e., a shift's domain becomes empty), backtrack to previous assignments and try alternative values.
   - **Store the Daily Schedule:**
     - Once all shifts for the day have been successfully assigned, store the day's schedule in the overall `weeklySchedule`.

2. **Return the Weekly Schedule:**
   - After processing every day, return the complete `weeklySchedule`.

# **Solution**

# Employee Shift Scheduling using CSP

In this notebook, we implement a weekly shift scheduling system for employees. The system uses Constraint Satisfaction Problem (CSP) techniques, including:
- **AC-3** for domain pruning (enforcing arc consistency)
- **MRV (Minimum Remaining Values)** to choose the next shift to assign
- **LCV (Least Constraining Value)** to order the employee options
- **Backtracking** to search for a valid complete assignment

Our constraint is simple: **No employee can work more than one shift per day.**

## Input Data

We define the list of employees, their availability (which days and shifts they can work), and the days and shifts we need to schedule.



In [7]:
# List of employees
employees = ["Alice", "Bob", "Carol", "Dave"]

# Employee availability dictionary
# Each employee is mapped to the days they are available, and for each day, the list of shifts they can work.
availability = {
    "Alice": {
        "Monday": ["Morning", "Afternoon", "Night"],
        "Wednesday": ["Morning", "Afternoon", "Night"],
        "Friday": ["Morning", "Afternoon", "Night"]
    },
    "Bob": {
        "Tuesday": ["Morning", "Afternoon", "Night"],
        "Thursday": ["Morning", "Afternoon", "Night"]
    },
    "Carol": {
        "Monday": ["Morning", "Afternoon"],
        "Tuesday": ["Morning"],
        "Wednesday": ["Afternoon", "Night"],
        "Thursday": ["Night"],
        "Friday": ["Morning", "Afternoon", "Night"]
    },
    "Dave": {
        "Monday": ["Night"],
        "Tuesday": ["Afternoon", "Night"],
        "Wednesday": ["Morning", "Afternoon"],
        "Thursday": ["Morning", "Afternoon", "Night"],
        "Friday": ["Afternoon"]
    }
}

# Define the days and shifts for the schedule (5 days, 3 shifts per day)
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday"]
shifts = ["Morning", "Afternoon", "Night"]




## Initializing the Daily Schedule and Domains

For each day, we build:
- **day_sched:** a dictionary mapping `(day, shift)` to an assigned employee (initially `None`).
- **day_domains:** a dictionary mapping `(day, shift)` to the list of employees available for that shift.

The `init_day_schedule` function does this.


In [1]:
def init_day_schedule(day, shift_list, people, avail):
    """
    Initializes the schedule and domain dictionaries for a given day.
    - day_sched: maps (day, shift) -> assigned employee (None initially)
    - day_domains: maps (day, shift) -> list of available employees
    """
    day_sched = {}   # e.g., {("Monday", "Morning"): None, ("Monday", "Afternoon"): None, ...}
    day_domains = {} # e.g., {("Monday", "Morning"): ["Alice", "Carol"], ...}
    
    for shift in shift_list:
        var = (day, shift)
        day_sched[var] = None  # No assignment yet.
        day_domains[var] = []  # Start with an empty list.
        for person in people:
            # If the person is available for this day and shift, add them.
            if day in avail.get(person, {}) and shift in avail[person][day]:
                day_domains[var].append(person)
    return day_sched, day_domains


## AC-3 Algorithm and Related Functions

**AC-3** is used to enforce arc consistency.  
- **Arc:** In our problem, an arc is a pair of shifts on the same day that must have different employees.
- The algorithm checks each arc, and if a value in one shift's domain is not supported by any different value in the connected shift's domain, it removes that value.

We have two functions:
- `get_all_arcs`: Returns a list of all arcs for the day.
- `revise`: Checks and removes unsupported values from a variable's domain.
- `AC3`: Processes the arcs to prune domains.


In [2]:
def get_all_arcs(day_domains, day):
    """
    Returns all arcs (pairs of shifts) for a given day.
    An arc is a pair of shifts that share the constraint "one employee per day."
    """
    shift_vars = [(day, shift) for shift in shifts]
    arcs = []
    for i in range(len(shift_vars)):
        for j in range(len(shift_vars)):
            if i != j:
                arcs.append((shift_vars[i], shift_vars[j]))
    return arcs

def revise(domains, Xi, Xj):
    """
    Revise the domain for Xi based on Xj.
    For each value in Xi's domain, if all values in Xj's domain equal that value,
    then remove that value from Xi's domain.
    
    Returns True if any value was removed.
    """
    changed = False
    for value in domains[Xi][:]:  # iterate over a copy
        # If every value in Xj's domain is equal to 'value', then there's no alternative.
        if all(value == other for other in domains[Xj]):
            domains[Xi].remove(value)
            changed = True
    return changed

def AC3(domains, schedule, day):
    """
    AC-3 algorithm to enforce arc consistency.
    It processes all arcs (pairs of shifts on the same day) and prunes domains.
    
    Returns True if all domains are consistent (non-empty), else False.
    """
    print("AC3 - Starting domain:")
    print(domains)
    print("Current schedule:")
    print(schedule)
    
    queue = get_all_arcs(domains, day)
    while queue:
        (Xi, Xj) = queue.pop(0)
        if revise(domains, Xi, Xj):
            if not domains[Xi]:
                return False  # No possible value for Xi, failure.
            # Re-check all neighbors of Xi (other shifts on the same day) since Xi's domain changed.
            for neighbor in [(day, s) for s in shifts if (day, s) != Xi and (day, s) != Xj]:
                if (neighbor, Xi) not in queue:
                    queue.append((neighbor, Xi))
    return True


## Heuristics and Consistency Check

We use two heuristics:
- **MRV (Minimum Remaining Values):** Select the unassigned shift with the fewest available employees.
- **LCV (Least Constraining Value):** Order the employee options by how few choices they eliminate from other shifts.

The function `is_consistent` checks that an employee is not assigned to more than one shift on the same day.


In [3]:
def select_unassigned_var(schedule, domains):
    """
    Selects an unassigned variable (shift) using the MRV heuristic.
    Returns the shift with the smallest domain (fewest options).
    """
    unassigned = [var for var, value in schedule.items() if value is None]
    print("MRV - Unassigned variables:")
    print(unassigned)
    
    if not unassigned:
        return None
    return min(unassigned, key=lambda var: len(domains[var]))

def order_domain_values(var, domains):
    """
    Orders the values in the domain of the variable using the LCV heuristic.
    It sorts the employees by how many values they would remove from neighboring shifts.
    Lower count means less constraining.
    """
    day, _ = var
    neighbors = [(day, s) for s in shifts if (day, s) != var]
    
    def count_eliminations(emp):
        elim = 0
        for neighbor in neighbors:
            if emp in domains[neighbor]:
                elim += 1
        return elim
    
    return sorted(domains[var], key=count_eliminations)

def is_consistent(var, value, schedule):
    """
    Checks whether assigning 'value' to 'var' is consistent with our constraint:
    An employee cannot work more than one shift per day.
    """
    day, _ = var
    for (d, s), assigned in schedule.items():
        if d == day and assigned == value:
            return False
    return True


## Backtracking Search

The `backtrack` function recursively assigns employees to each shift for a day. It uses forward checking to remove an assigned employee from the domains of other shifts on that day, then calls AC-3 to propagate the constraints. If a conflict occurs later, it backtracks (undoes the assignment) and tries a different value.


In [4]:
def backtrack(schedule, domains, day):
    """
    Recursively tries to assign employees to all shifts in a day using backtracking.
    Returns a complete schedule for the day, or None if no valid assignment exists.
    """
    # If every shift is assigned, the schedule is complete.
    if all(val is not None for val in schedule.values()):
        return schedule
    
    # Select an unassigned shift using MRV.
    shift_var = select_unassigned_var(schedule, domains)
    
    # Try each employee option for this shift, ordered by LCV.
    for emp in order_domain_values(shift_var, domains):
        if is_consistent(shift_var, emp, schedule):
            # Tentatively assign the employee.
            schedule[shift_var] = emp
            
            # Save a copy of the current domains for backtracking.
            saved_domains = {v: domains[v][:] for v in domains}
            
            # Forward checking: remove the assigned employee from other shifts on the same day.
            for other in [v for v in schedule if v[0] == day and v != shift_var]:
                if emp in domains[other]:
                    domains[other].remove(emp)
            
            # Propagate constraints with AC-3.
            if AC3(domains, schedule, day):
                result = backtrack(schedule, domains, day)
                if result is not None:
                    return result
            
            # Backtrack: undo the assignment and restore the previous domains.
            schedule[shift_var] = None
            domains = saved_domains
    return None  # No valid assignment found.


## Weekly Scheduling

The `schedule_weekly_shifts` function processes each day separately. For each day, it initializes the day's schedule and domains, applies AC-3 to prune domains, and then uses backtracking to find a valid assignment. Finally, it collects each day's schedule into a full weekly schedule.


In [5]:
def schedule_weekly_shifts(employees, availability, days, shifts):
    """
    Schedules shifts for each day of the week using CSP, AC-3, and backtracking.
    Returns a dictionary with the complete weekly schedule.
    """
    week_sched = {}
    
    for current_day in days:
        print("\nScheduling for:", current_day)
        day_sched, day_domains = init_day_schedule(current_day, shifts, employees, availability)
        
        # Run AC-3 to prune domains for the day.
        if not AC3(day_domains, day_sched, current_day):
            print("No solution exists for", current_day)
            return None
        
        # Use backtracking to assign employees to shifts.
        result = backtrack(day_sched, day_domains, current_day)
        if result is None:
            print("Backtracking failed for", current_day)
            return None
        
        week_sched[current_day] = result
    return week_sched


## Run the Scheduler

The final step is to run the scheduler and print out the weekly schedule.


In [9]:
# Run the scheduler and print the weekly schedule

final_schedule = schedule_weekly_shifts(employees, availability, days, shifts)
if final_schedule:
    for day, sched in final_schedule.items():
        print("\n" + day + ":")
        for (d, shift), person in sorted(sched.items()):
            print(f"  {shift}: {person}")
else:
    print("No valid schedule found.")



Scheduling for: Monday
AC3 - Starting domain:
{('Monday', 'Morning'): ['Alice', 'Carol'], ('Monday', 'Afternoon'): ['Alice', 'Carol'], ('Monday', 'Night'): ['Alice', 'Dave']}
Current schedule:
{('Monday', 'Morning'): None, ('Monday', 'Afternoon'): None, ('Monday', 'Night'): None}
MRV - Unassigned variables:
[('Monday', 'Morning'), ('Monday', 'Afternoon'), ('Monday', 'Night')]
AC3 - Starting domain:
{('Monday', 'Morning'): ['Alice', 'Carol'], ('Monday', 'Afternoon'): ['Alice'], ('Monday', 'Night'): ['Alice', 'Dave']}
Current schedule:
{('Monday', 'Morning'): 'Carol', ('Monday', 'Afternoon'): None, ('Monday', 'Night'): None}
MRV - Unassigned variables:
[('Monday', 'Afternoon'), ('Monday', 'Night')]
AC3 - Starting domain:
{('Monday', 'Morning'): ['Carol'], ('Monday', 'Afternoon'): ['Alice'], ('Monday', 'Night'): ['Dave']}
Current schedule:
{('Monday', 'Morning'): 'Carol', ('Monday', 'Afternoon'): 'Alice', ('Monday', 'Night'): None}
MRV - Unassigned variables:
[('Monday', 'Night')]
AC3 - 